<a href="https://colab.research.google.com/github/wonmadeit/LEEGAWON/blob/main/9_Sequential_method_to_build_a_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Sequential class를 활용한 간편한 신경망 정의

In [1]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self):
        return len(self.x)

In [ ]:
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

- `nn` 패키지의 `Sequential` method를 사용하여 신경망 모형 정의

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

- torch_summary를 설치하여, 모형을 요약하는 함수 `summary`를 사용하여 정의한 모형 확인

In [ ]:
!pip install torch_summary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


- 출력 결과 해석
  - `Output Shape`의 `-1`의미 : 정해진 숫자가 아닌, batch size 만큼 구성될 수 있음을 의미
  - `Output Shape`의 `8`의미 : 해당 레이어를 통과한 Output은 차원의 크기가 `8`임
  - `Param #` : parameter 개수 (24= 2x8weights+8bias    9 = 8wiehgts + 1bias)
  - `Total mult-adds` : operation 개수
  - `Trainable params` : 학습 가능한 parameter 수


In [ ]:
summary(model, torch.zeros(1,2));   #torch.zeros(1,2)= 예시 input

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [ ]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)
import time
loss_history = []
start = time.time()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.17645740509033203


In [ ]:
val = [[8,9],[10,11],[1.5,2.5]]
val = torch.tensor(val).float()

In [ ]:
model(val.to(device))

tensor([[16.8142],
        [20.6777],
        [ 4.2581]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
val.sum(-1)

tensor([17., 21.,  4.])